## MSDS 7331: Data Mining

## Mini Lab: Logistic Regression and SVMs

## 9 June 2019

## Authors: Meredith Ludlow, Anand Rajan, Kristen Rollins, and Tej Tenmattam

---

### Create Models

<div class="alert alert-block alert-info">
<b>Rubric 1:</b> Create a logistic regression model and a support vector machine model for the classification task involved with your dataset. Assess how well each model performs (use 80/20 training/testing split for your data). Adjust parameters of the models to make them more accurate. If your dataset size requires the use of stochastic gradient descent, then linear kernel only is fine to use. That is, the SGDClassifier is fine to use for optimizing logistic regression and linear support vector machines. For many problems, SGD will be required in order to train the SVM model in a reasonable timeframe.
</div>

In [1]:
# importing necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Set seaborn plot styles
sns.set_style('darkgrid')
sns.set_color_codes('muted')

#uncomment when ready to turn in report
#import warnings
#warnings.filterwarnings("ignore") # ignore warnings for clean report

# df.head() displays all the columns without truncating
pd.set_option('display.max_columns', None)

# read csv file as pandas dataframe
df_17_census = pd.read_csv('Data/acs2017_census_tract_data.csv')

In [2]:
# Clean dataset as in lab 1
df_17_census.set_index('TractId', inplace=True) # set tract id as index

#Drop tracts where population is 0
df_17_cln = df_17_census.drop(df_17_census[df_17_census.TotalPop == 0].index)

#Drop tracts where child poverty or unemployment is null
df_17_cln = df_17_cln[np.isfinite(df_17_cln['ChildPoverty'])]
df_17_cln = df_17_cln[np.isfinite(df_17_cln['Unemployment'])]

#Impute to the median by each state
df_grouped = df_17_cln.groupby('State').transform(lambda x: x.fillna(x.median()))
df_17_cln['Income'] = df_grouped['Income']
df_17_cln['IncomeErr'] = df_grouped['IncomeErr']

#Impute remaining values to the overall median
df_17_cln = df_17_cln.fillna(df_17_cln.median())

In [3]:
# Categorize the unemployed percentages
# Make cutoffs using quartiles of clean dataset, so they are roughly equal
df_17_cln['Unemployment_Cat'] = pd.cut(df_17_cln.Unemployment,[-1,3.9,6,9,101],labels=['low','med','high','very_high'])

# Also cut unemployment into binary categories if it's easier to work with
df_17_cln['Unemployment_Binary'] = pd.cut(df_17_cln.Unemployment,[-1,6,101],labels=['low','high'])

df_17_cln.info() # matches cleaned dataset from lab 1

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72889 entries, 1001020100 to 72153750602
Data columns (total 38 columns):
State                  72889 non-null object
County                 72889 non-null object
TotalPop               72889 non-null int64
Men                    72889 non-null int64
Women                  72889 non-null int64
Hispanic               72889 non-null float64
White                  72889 non-null float64
Black                  72889 non-null float64
Native                 72889 non-null float64
Asian                  72889 non-null float64
Pacific                72889 non-null float64
VotingAgeCitizen       72889 non-null int64
Income                 72889 non-null float64
IncomeErr              72889 non-null float64
IncomePerCap           72889 non-null float64
IncomePerCapErr        72889 non-null float64
Poverty                72889 non-null float64
ChildPoverty           72889 non-null float64
Professional           72889 non-null float64
Service     

### Model Advantages

<div class="alert alert-block alert-info">
<b>Rubric 2:</b> Discuss the advantages of each model for each classification task. Does one type of model offer superior performance over another in terms of prediction accuracy? In terms of training time or efficiency? Explain in detail.
</div>

### Interpret Feature Importance

<div class="alert alert-block alert-info">
<b>Rubric 3:</b> Use the weights from logistic regression to interpret the importance of different features for the classification task. Explain your interpretation in detail. Why do you think some variables are more important?
</div>

### Interpret Support Vectors

<div class="alert alert-block alert-info">
<b>Rubric 4:</b> Look at the chosen support vectors for the classification task. Do these provide any insight into the data? Explain. If you used stochastic gradient descent (and therefore did not explicitly solve for support vectors), try subsampling your data to train the SVC model— then analyze the support vectors from the subsampled dataset.
</div>